Estandarizacion

In [1]:
import math
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from keras.layers import Dropout
from keras.layers import BatchNormalization
import requests
from bs4 import BeautifulSoup
import os
from astropy.io import fits
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table
from tqdm import tqdm
import time
from glob import glob
from astropy.utils.data import get_pkg_data_filename
from astropy.convolution import convolve, Gaussian1DKernel

Aqui creamos un vector, lista de listas, con cada uno de los espectros. Hay tres vectores, uno para R, B y Z

In [19]:
R=[]# en estas listas se guardaran todos los espectros
Z=[]
B=[]

archivos= ['DataDESI_50.fits', 'DataDESI_100.fits', 'DataDESI_150.fits', 'DataDESI_200.fits', 'DataDESI_250.fits',
           'DataDESI600-699.fits', 'DataDESI700-758.fits']

for h in range(len(archivos)):
    espc = fits.open(archivos[h]) #open file
    
    for i in range(1,len(espc)): # Solo escogemos las primeras 2 tablas de los fits DataDESI_50 y DataDESI_100
        Tabla_fit= Table.read(espc, hdu=i) #hdu=i donde i va desde 1 hasta 5
        
        for j in range(len(Tabla_fit['BFlux'])):
            RFlux = [[float(k)] for k in Tabla_fit['RFlux'][j].split(',')]  # Convierte la cadena a una lista de float
            ZFlux = [[float(l)] for l in Tabla_fit['ZFlux'][j].split(',')]
            BFlux = [[float(m)] for m in Tabla_fit['BFlux'][j].split(',')]
    
            RFlux = np.array(RFlux)
            ZFlux = np.array(ZFlux)
            BFlux = np.array(BFlux)
    
            R.append(RFlux) # Agrega la lista de números de punto flotante a los datos. Estas son las listas de flujo divididas
            Z.append(ZFlux)
            B.append(BFlux)
        

51
51
51
51
51
49
28


Ahora creamos dos listas, en una estan los promedios de cada longitud de onda y en la otra las desviaciones estandar.

In [28]:
def calcular_media_y_desviacion_de_lista_de_listas(lista_de_listas):
    # Obtener la longitud de la lista de listas y la cantidad de columnas
    num_filas = len(lista_de_listas)
    num_columnas = len(lista_de_listas[0])

    # Inicializar listas para almacenar los resultados de la media y la desviación estándar
    media = [0] * num_columnas
    desviacion_estandar = [0] * num_columnas

    # Calcular la media y la desviación estándar columna por columna
    for fila in lista_de_listas:
        for i in range(num_columnas):
            media[i] += fila[i]

    for i in range(num_columnas):
        media[i] /= num_filas

    for fila in lista_de_listas:
        for i in range(num_columnas):
            desviacion_estandar[i] += (fila[i] - media[i]) ** 2

    for i in range(num_columnas):
        desviacion_estandar[i] = math.sqrt(desviacion_estandar[i] / num_filas)

    return media, desviacion_estandar

media_R, desv_R = calcular_media_y_desviacion_de_lista_de_listas(R)
media_Z, desv_Z = calcular_media_y_desviacion_de_lista_de_listas(Z)
media_B, desv_B = calcular_media_y_desviacion_de_lista_de_listas(B)

In [2]:
archivos= ['DataDESI_50.fits', 'DataDESI_100.fits', 'DataDESI_150.fits', 'DataDESI_200.fits', 'DataDESI_250.fits',
           'DataDESI600-699.fits', 'DataDESI700-758.fits']

for h in range(len(archivos)):
    espc = fits.open(archivos[h]) #open file
    espc.info()

Filename: DataDESI_50.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     20   41R x 6C   [100000A, 100000A, 100000A, 10A, D, J]   
  2                1 BinTableHDU     20   5R x 6C   [100000A, 100000A, 100000A, 10A, D, J]   
  3                1 BinTableHDU     20   139R x 6C   [100000A, 100000A, 100000A, 10A, D, J]   
  4                1 BinTableHDU     20   2804R x 6C   [100000A, 100000A, 100000A, 10A, D, J]   
  5                1 BinTableHDU     20   111R x 6C   [100000A, 100000A, 100000A, 10A, D, J]   
  6                1 BinTableHDU     20   3978R x 6C   [100000A, 100000A, 100000A, 10A, D, J]   
  7                1 BinTableHDU     20   530R x 6C   [100000A, 100000A, 100000A, 10A, D, J]   
  8                1 BinTableHDU     20   2192R x 6C   [100000A, 100000A, 100000A, 10A, D, J]   
  9                1 BinTableHDU     20   4123R x 6C   [100000A, 100000A, 100000A, 10A, D, J